In [1]:
from ipynb.fs.full.gbs import *

## Stochastic Tests

Tests are not currently comprehensive nor rigorous. Outputs need to be checked by eye.

TESTS ARE ABLE TO RUN != CODE IS WORKING CORRECTLY

### test `zpf`

In [2]:
def check_expectations(arr: np.ndarray):
    # avg z
    exp_val = arr.mean()

    # avg z^T z for every column z in arr
    # if statement optimizes speed
    trans = np.trace((arr @ arr.T if arr.shape[0] < arr.shape[1] else arr.T @ arr)) / arr.shape[1]

    # avg z^H z for every column z in arr
    herm = np.linalg.norm(arr, axis=0).mean()
    
    print(f"E[z] = {exp_val} should be 0.")
    print(f"E[z^T z] = {trans} should be 0.")
    print(f"E[z^H z] = {herm} should be 1.")

In [3]:
vac = zpf(1000000, 2)
check_expectations(vac)

E[z] = (-0.0002866807283946047+0.0006293897116304567j) should be 0.
E[z^T z] = (-0.0003384787341390289-0.0005729062371896246j) should be 0.
E[z^H z] = 0.9399600919308655 should be 1.


### test `threshold_detector` and `h_or_v_detector`

In [4]:
a = zpf(100,2)

#print(a)

td = threshold_detector(a, 1)
print(td)

hv = h_or_v_detector(a, 1)
print(hv)

td_any = td.any(axis=0)
assert(np.array_equal(hv, td_any))

[[False False False False  True False False False False False False False
  False False False False False False False False False False False False
  False False False False False  True  True False False False False False
  False False False False False False False False False False False False
  False False False  True False False False False False False False False
   True False False False False False False False  True False False False
   True False False  True False False False False False False False False
  False False False False False False False False  True False False False
  False False False False]
 [False False False False False False False  True  True False False False
  False False False False False False  True False False False  True False
  False False False  True False False False False False False False False
  False False False False False False  True False False False False  True
   True False False False False False False False False  True False False
  False Fal

### test `ent`, `zpf`, `pbs`, coincidence counters

In [5]:
l,r = ent(1000, 2)
lzpf = zpf(1000, 2)
rzpf = zpf(1000, 2)

In [6]:
lt, lr = pbs(l, lzpf)
rt, rr = pbs(r, rzpf)

dlt, dlr, drt, drr = map(h_or_v_detector, (lt, lr, rt, rr))
detections = np.array([dlt, dlr, drt, drr])

coincidence_counts = get_all_coincidence_counts(detections)
dlt_and_drt = sum([coincidence_counts[event] for event in [(1,0,1,0), (1,0,1,1), (1,1,1,0), (1,1,1,1)]])
assert(get_coincidence_count(detections[[0,2],]) == dlt_and_drt)

print_all_coincidence_counts(detections)


0000 61
0001 2
0010 1
0011 0
0100 2
0101 165
0110 0
0111 7
1000 1
1001 1
1010 176
1011 7
1100 0
1101 3
1110 9
1111 565


## QM Tests

Tests are not currently comprehensive nor rigorous. Outputs need to be checked by eye.

TESTS ARE ABLE TO RUN != CODE IS WORKING CORRECTLY

### test `get_fock_basis_states`

Takes a long time and eats up memory very quickly. The advantage of using `@cache` can be seen by running `get_fock_basis_states` twice in a row, or running it with smaller inputs after it finishes - after the first run, future calls are almost instantaneous.

In [7]:
# get_fock_basis_states(3,100)

### test `qr_haar` and `HilbertSpaceUnitary`

In [8]:
U = qr_haar(100) #np.array([[0, 1], [1, 0]], dtype=complex)

#print(U)

UH = HilbertSpaceUnitary(U, 1)

zers = np.zeros(100, dtype=int)
zert = np.zeros(100, dtype=int)

zers[0] = 1
zert[0] = 1

print(abs(UH.get_entry(zers, zert))**2)
print(abs(UH[zers, zert])**2)

0.02649127698658158
0.02649127698658158


### test `get_hilbert_space_unitary_matrix`

In [9]:
hadamard = np.array([[1, 1], [1, -1]]) / sqrt(2)

print("Hadamard with 1 photon, 2 modes:")
hsu_had = get_hilbert_space_unitary_matrix(hadamard, 1, p=True) # p=True prints resulting unitary
print()

print("Hadamard with 2 photons, 2 modes:")
hsu_had = get_hilbert_space_unitary_matrix(hadamard, 2, p=True)
print()

m = 6
U = qr_haar(m)

print(f"Haar random unitary with 1 photon, {m} modes:")
hsu_haar = get_hilbert_space_unitary_matrix(U, 1, p=True)
print()

print(f"Haar random unitary with 2 photons, {m} modes:")
hsu_haar = get_hilbert_space_unitary_matrix(U, 2, p=True)
print()

Hadamard with 1 photon, 2 modes:
[1 0]:	[0.71-0.j 0.71-0.j]
[0 1]:	[ 0.71-0.j -0.71+0.j]

Hadamard with 2 photons, 2 modes:
[0 2]:	[ 0.5 +0.j -0.71+0.j  0.5 +0.j]
[1 1]:	[-0.71+0.j  0.  +0.j  0.71+0.j]
[2 0]:	[0.5 +0.j 0.71+0.j 0.5 +0.j]

Haar random unitary with 1 photon, 6 modes:
[1 0 0 0 0 0]:	[ 0.3 +0.11j  0.4 -0.4j  -0.08+0.64j -0.18-0.08j  0.02+0.32j -0.16-0.03j]
[0 1 0 0 0 0]:	[ 0.46-0.19j  0.14-0.21j -0.1 -0.14j  0.35+0.54j -0.18-0.15j  0.2 -0.39j]
[0 0 1 0 0 0]:	[ 0.57-0.03j -0.41+0.41j -0.08+0.3j   0.16-0.16j -0.13+0.07j  0.25+0.3j ]
[0 0 0 1 0 0]:	[ 0.34-0.08j  0.25+0.01j  0.29-0.26j -0.14-0.22j -0.47-0.33j -0.46+0.22j]
[0 0 0 0 1 0]:	[ 0.19+0.09j  0.07+0.38j  0.09-0.24j  0.1 +0.34j  0.27+0.55j -0.5 +0.03j]
[0 0 0 0 0 1]:	[-0.15-0.37j -0.12+0.22j -0.34+0.36j -0.33+0.44j -0.01-0.34j -0.34+0.08j]

Haar random unitary with 2 photons, 6 modes:
[0 0 0 0 0 2]:	[ 0.11-0.05j  0.04+0.16j -0.11+0.01j  0.22+0.15j  0.11-0.25j -0.09-0.29j
 -0.07-0.37j  0.18+0.15j  0.12-0.21j -0.02-0.24j 